**LIBRARY**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import plotly.graph_objects as go
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

**LOAD DATASET**

In [ ]:
bbca = pd.read_csv('BBCA_clean_modified.csv')
bbni = pd.read_csv('BBNI_clean_modified.csv')
bbri = pd.read_csv('BBRI_clean_modified.csv')
bmri = pd.read_csv('BMRI_clean_modified.csv')

In [ ]:
def preprocess_data(df):
    df = df.dropna().reset_index(drop=True)
    features = df[['Open', 'High', 'Low', 'Volume', 'H-L', 'O-C', '7 DAYS MA', '14 DAYS MA', '21 DAYS MA', '7 DAYS STD DEV']]
    target = df['Close']
    scaler = MinMaxScaler()
    features = scaler.fit_transform(features)
    return train_test_split(features, target, test_size=0.2, random_state=42), df['Date']

In [ ]:
def ann_model(input_dim):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(input_dim,)))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

In [ ]:
def ann_model(input_dim):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(input_dim,)))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

# Membuat model dengan input_dim sesuai kebutuhan
input_dim = 500  # Contoh: asumsi bahwa kita memiliki 10 fitur input
model = ann_model(input_dim)

# Mencetak rangkuman model
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 128)               64128     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 74497 (291.00 KB)
Trainable params: 7449

In [ ]:
def hyperparameter_tuning(model_builder, X_train, y_train, X_val, y_val):
    best_params = None
    best_score = float('inf')
    param_grid = {
        'batch_size': [16, 32],
        'epochs': [50, 100]
    }

    for batch_size in param_grid['batch_size']:
        for epochs in param_grid['epochs']:
            model = model_builder(X_train.shape[1])
            history = model.fit(X_train, y_train,
                                epochs=epochs,
                                batch_size=batch_size,
                                validation_data=(X_val, y_val),
                                callbacks=[EarlyStopping(patience=10)],
                                verbose=0)
            score = np.min(history.history['val_loss'])
            if score < best_score:
                best_score = score
                best_params = {'batch_size': batch_size, 'epochs': epochs}
    print(f"Best: {best_score} using {best_params}")
    return best_params

# Evaluate Model with Cross-Validation
def evaluate_model_with_cv(model_builder, X_train, X_test, y_train, y_test, model_name):
    best_params = hyperparameter_tuning(model_builder, X_train, y_train, X_test, y_test)
    model = model_builder(X_train.shape[1])
    checkpoint = ModelCheckpoint(f'{model_name}.h5', monitor='val_loss', save_best_only=True, verbose=0)
    history = model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'], validation_data=(X_test, y_test),
                        callbacks=[EarlyStopping(patience=10), checkpoint], verbose=1)
    model = load_model(f'{model_name}.h5')

    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)

    train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
    test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
    train_mae = mean_absolute_error(y_train, train_predictions)
    test_mae = mean_absolute_error(y_test, test_predictions)

    return train_rmse, test_rmse, train_mae, test_mae, train_predictions, test_predictions

In [ ]:
def visualize_model_performance(dates, y_train, y_test, y_train_pred, y_test_pred, title):
    train_dates = dates[:len(y_train)]
    test_dates = dates[len(y_train):len(y_train) + len(y_test)]
    all_dates = dates[:len(y_train) + len(y_test)]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=train_dates, y=y_train, mode='markers', name='Train Actual', marker=dict(color='gray')))
    fig.add_trace(go.Scatter(x=test_dates, y=y_test, mode='markers', name='Test Actual', marker=dict(color='red')))

    # Combine train and test predictions for a single prediction line
    all_predictions = np.concatenate((y_train_pred, y_test_pred))

    fig.add_trace(go.Scatter(x=all_dates, y=all_predictions.flatten(), mode='lines', name='Predictions', line=dict(color='orange', width=2, dash='dot')))

    fig.update_layout(
        title=title,
        xaxis=dict(
            rangeslider=dict(visible=True),
            type='date'
        ),
        yaxis=dict(title='Close Price IDR'),
        xaxis_range=[all_dates.iloc[0], all_dates.iloc[-1]]
    )
    fig.show()

In [ ]:
def process_dataset(df, dataset_name):
    (X_train, X_test, y_train, y_test), dates = preprocess_data(df)
    X_train_dl = X_train
    X_test_dl = X_test

    ann_train_rmse, ann_test_rmse, ann_train_mae, ann_test_mae, ann_train_predictions, ann_test_predictions = evaluate_model_with_cv(ann_model, X_train_dl, X_test_dl, y_train, y_test, f'{dataset_name}_ANN')

    visualize_model_performance(dates, y_train, y_test, ann_train_predictions, ann_test_predictions, f'Enhanced ANN Model for {dataset_name}')

    return {
        'Dataset': dataset_name,
        'ANN_Train_RMSE': ann_train_rmse,
        'ANN_Test_RMSE': ann_test_rmse,
        'ANN_Train_MAE': ann_train_mae,
        'ANN_Test_MAE': ann_test_mae
    }

In [ ]:
results=[]

In [ ]:
BBCARES=process_dataset(bbca, 'BBCA')

Best: 3263.899658203125 using {'batch_size': 16, 'epochs': 100}
Epoch 1/100
62/62 [==============================] - 1s 6ms/step - loss: 16757587.0000 - val_loss: 16872534.0000
Epoch 2/100
51/62 [=======================>......] - ETA: 0s - loss: 15398450.0000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


62/62 [==============================] - 0s 3ms/step - loss: 14826263.0000 - val_loss: 11470473.0000
Epoch 3/100
62/62 [==============================] - 0s 3ms/step - loss: 4847870.0000 - val_loss: 140465.5312
Epoch 4/100
62/62 [==============================] - 0s 3ms/step - loss: 286004.4062 - val_loss: 69503.4297
Epoch 5/100
62/62 [==============================] - 0s 3ms/step - loss: 267411.4062 - val_loss: 64232.8711
Epoch 6/100
62/62 [==============================] - 0s 5ms/step - loss: 227379.2500 - val_loss: 62229.0703
Epoch 7/100
62/62 [==============================] - 0s 3ms/step - loss: 224148.9062 - val_loss: 55515.3203
Epoch 8/100
62/62 [==============================] - 0s 4ms/step - loss: 236130.8281 - val_loss: 52646.5117
Epoch 9/100
62/62 [==============================] - 0s 4ms/step - loss: 238648.1562 - val_loss: 47644.7109
Epoch 10/100
62/62 [==============================] - 0s 3ms/step - loss: 234344.3906 - val_loss: 48435.4648
Epoch 11/100
62/62 [============

In [ ]:
results.append(BBCARES)
pd.DataFrame(BBCARES, index=[0]).T

,0
Dataset,BBCA
ANN_Train_RMSE,56.128042
ANN_Test_RMSE,53.834626
ANN_Train_MAE,44.059112
ANN_Test_MAE,41.991976


In [ ]:
BBNIRES=process_dataset(bbni, 'BBNI')

Best: 4662.5634765625 using {'batch_size': 16, 'epochs': 50}
Epoch 1/50
62/62 [==============================] - 1s 8ms/step - loss: 18001962.0000 - val_loss: 17287234.0000
Epoch 2/50
48/62 [======================>.......] - ETA: 0s - loss: 17134306.0000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



62/62 [==============================] - 0s 5ms/step - loss: 16684969.0000 - val_loss: 13403477.0000
Epoch 3/50
62/62 [==============================] - 0s 6ms/step - loss: 7374036.0000 - val_loss: 795576.9375
Epoch 4/50
62/62 [==============================] - 0s 5ms/step - loss: 322094.2500 - val_loss: 75490.5234
Epoch 5/50
62/62 [==============================] - 0s 5ms/step - loss: 262366.9375 - val_loss: 72888.1797
Epoch 6/50
62/62 [==============================] - 0s 7ms/step - loss: 266952.1875 - val_loss: 67823.4922
Epoch 7/50
62/62 [==============================] - 0s 6ms/step - loss: 248837.2656 - val_loss: 64194.3203
Epoch 8/50
62/62 [==============================] - 0s 5ms/step - loss: 240820.1094 - val_loss: 69591.0703
Epoch 9/50
62/62 [==============================] - 0s 7ms/step - loss: 247548.3438 - val_loss: 60356.2812
Epoch 10/50
62/62 [==============================] - 0s 5ms/step - loss: 240625.7656 - val_loss: 54789.4062
Epoch 11/50
62/62 [=====================

In [ ]:
results.append(BBNIRES)
pd.DataFrame(BBNIRES, index=[0]).T

,0
Dataset,BBNI
ANN_Train_RMSE,79.800934
ANN_Test_RMSE,76.133826
ANN_Train_MAE,59.96416
ANN_Test_MAE,57.663513


In [ ]:
BBRIRES=process_dataset(bbri, 'BBRI')

Best: 2702.88037109375 using {'batch_size': 16, 'epochs': 100}
Epoch 1/100
62/62 [==============================] - 2s 9ms/step - loss: 15084336.0000 - val_loss: 14735315.0000
Epoch 2/100
52/62 [========================>.....] - ETA: 0s - loss: 13526788.0000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



62/62 [==============================] - 0s 5ms/step - loss: 13149748.0000 - val_loss: 9475593.0000
Epoch 3/100
62/62 [==============================] - 0s 5ms/step - loss: 3809958.7500 - val_loss: 77972.3438
Epoch 4/100
62/62 [==============================] - 0s 5ms/step - loss: 210058.5312 - val_loss: 56545.4023
Epoch 5/100
62/62 [==============================] - 0s 5ms/step - loss: 205855.5938 - val_loss: 55123.5430
Epoch 6/100
62/62 [==============================] - 0s 6ms/step - loss: 223433.5781 - val_loss: 48634.3047
Epoch 7/100
62/62 [==============================] - 0s 5ms/step - loss: 199286.6250 - val_loss: 51759.8750
Epoch 8/100
62/62 [==============================] - 0s 5ms/step - loss: 194283.9062 - val_loss: 45118.4023
Epoch 9/100
62/62 [==============================] - 0s 6ms/step - loss: 193329.2812 - val_loss: 37766.7461
Epoch 10/100
62/62 [==============================] - 0s 5ms/step - loss: 205321.9531 - val_loss: 38211.6133
Epoch 11/100
62/62 [==============

In [ ]:
results.append(BBRIRES)
pd.DataFrame(BBRIRES, index=[0]).T

,0
Dataset,BBRI
ANN_Train_RMSE,53.927128
ANN_Test_RMSE,50.100568
ANN_Train_MAE,39.658694
ANN_Test_MAE,38.064357


In [ ]:
BMRIRES=process_dataset(bmri, 'BMRI')

Best: 4508.1708984375 using {'batch_size': 16, 'epochs': 100}
Epoch 1/100
62/62 [==============================] - 1s 6ms/step - loss: 53539716.0000 - val_loss: 51924668.0000
Epoch 2/100
48/62 [======================>.......] - ETA: 0s - loss: 51509780.0000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



62/62 [==============================] - 0s 4ms/step - loss: 50115428.0000 - val_loss: 42211000.0000
Epoch 3/100
62/62 [==============================] - 0s 3ms/step - loss: 26412610.0000 - val_loss: 6983682.5000
Epoch 4/100
62/62 [==============================] - 0s 3ms/step - loss: 2554388.5000 - val_loss: 1078681.0000
Epoch 5/100
62/62 [==============================] - 0s 4ms/step - loss: 1618183.3750 - val_loss: 975196.4375
Epoch 6/100
62/62 [==============================] - 0s 4ms/step - loss: 1471152.2500 - val_loss: 843289.2500
Epoch 7/100
62/62 [==============================] - 0s 4ms/step - loss: 1370557.2500 - val_loss: 724149.9375
Epoch 8/100
62/62 [==============================] - 0s 3ms/step - loss: 1170942.7500 - val_loss: 644582.4375
Epoch 9/100
62/62 [==============================] - 0s 3ms/step - loss: 1066656.5000 - val_loss: 540291.6250
Epoch 10/100
62/62 [==============================] - 0s 4ms/step - loss: 1022138.6250 - val_loss: 461508.6250
Epoch 11/100
62

In [ ]:
results.append(BMRIRES)
pd.DataFrame(BMRIRES, index=[0]).T

,0
Dataset,BMRI
ANN_Train_RMSE,65.914533
ANN_Test_RMSE,67.414694
ANN_Train_MAE,49.926418
ANN_Test_MAE,52.119016


In [ ]:
resultsall=pd.DataFrame(results).T
resultsall

,0,1,2,3
Dataset,BBCA,BBNI,BBRI,BMRI
ANN_Train_RMSE,56.128042,79.800934,53.927128,65.914533
ANN_Test_RMSE,53.834626,76.133826,50.100568,67.414694
ANN_Train_MAE,44.059112,59.96416,39.658694,49.926418
ANN_Test_MAE,41.991976,57.663513,38.064357,52.119016


In [ ]:
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

def hyperparameter_tuning(model_builder, X_train, y_train, X_val, y_val):
    best_params = None
    best_score = float('inf')
    param_grid = {
        'batch_size': [16, 32, 64],
        'epochs': [50, 100, 150],
        'learning_rate': [0.001, 0.005, 0.01]
    }
    for batch_size in param_grid['batch_size']:
        for epochs in param_grid['epochs']:
            for lr in param_grid['learning_rate']:
                model = model_builder(X_train.shape[1], lr)
                history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val),
                                    callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=0)
                score = np.min(history.history['val_loss'])
                if score < best_score:
                    best_score = score
                    best_params = {'batch_size': batch_size, 'epochs': epochs, 'learning_rate': lr}
    print(f"Best: {best_score} using {best_params}")
    return best_params

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

def ann_model(input_dim, learning_rate=0.001):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(input_dim,)))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    return model

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model

def evaluate_model_with_cv(model_builder, X_train, X_test, y_train, y_test, model_name):
    best_params = hyperparameter_tuning(model_builder, X_train, y_train, X_test, y_test)
    model = model_builder(X_train.shape[1], best_params['learning_rate'])
    checkpoint = ModelCheckpoint(f'{model_name}.h5', monitor='val_loss', save_best_only=True, verbose=0)
    history = model.fit(X_train, y_train, epochs=best_params['epochs'],
                        batch_size=best_params['batch_size'], validation_data=(X_test, y_test),
                        callbacks=[EarlyStopping(monitor='val_loss', patience=10), checkpoint], verbose=1)
    model = load_model(f'{model_name}.h5')
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)
    train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
    test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
    train_mae = mean_absolute_error(y_train, train_predictions)
    test_mae = mean_absolute_error(y_test, test_predictions)
    return train_rmse, test_rmse, train_mae, test_mae, train_predictions, test_predictions

In [ ]:
BMRIRES2=process_dataset(bmri, 'BMRI')

Best: 1959.862548828125 using {'batch_size': 16, 'epochs': 150, 'learning_rate': 0.005}
Epoch 1/150
62/62 [==============================] - 1s 6ms/step - loss: 38840548.0000 - val_loss: 2303454.2500
Epoch 2/150
51/62 [=======================>......] - ETA: 0s - loss: 2055595.6250

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



62/62 [==============================] - 0s 3ms/step - loss: 2002511.2500 - val_loss: 762280.5000
Epoch 3/150
62/62 [==============================] - 0s 4ms/step - loss: 1091582.6250 - val_loss: 407480.4375
Epoch 4/150
62/62 [==============================] - 0s 3ms/step - loss: 921921.4375 - val_loss: 192358.5469
Epoch 5/150
62/62 [==============================] - 0s 4ms/step - loss: 791711.1250 - val_loss: 251779.9531
Epoch 6/150
62/62 [==============================] - 0s 3ms/step - loss: 715075.5625 - val_loss: 79741.1641
Epoch 7/150
62/62 [==============================] - 0s 3ms/step - loss: 678748.2500 - val_loss: 57395.5742
Epoch 8/150
62/62 [==============================] - 0s 3ms/step - loss: 653327.6250 - val_loss: 41263.4219
Epoch 9/150
62/62 [==============================] - 0s 4ms/step - loss: 638551.0000 - val_loss: 34655.1406
Epoch 10/150
62/62 [==============================] - 0s 3ms/step - loss: 629815.3750 - val_loss: 97134.2500
Epoch 11/150
62/62 [=============

In [ ]:
results2=[]

In [ ]:
results.append(BMRIRES2)
pd.DataFrame(BMRIRES2, index=[0]).T

,0
Dataset,BMRI
ANN_Train_RMSE,37.259705
ANN_Test_RMSE,34.430958
ANN_Train_MAE,27.342167
ANN_Test_MAE,25.485461


In [ ]:
results2.append(BMRIRES2)
pd.DataFrame(BMRIRES2, index=[0]).T

,0
Dataset,BMRI
ANN_Train_RMSE,37.259705
ANN_Test_RMSE,34.430958
ANN_Train_MAE,27.342167
ANN_Test_MAE,25.485461


In [ ]:
BBRIRES2=process_dataset(bbri, 'BBRI')

Best: 1371.1865234375 using {'batch_size': 32, 'epochs': 100, 'learning_rate': 0.005}
Epoch 1/100
31/31 [==============================] - 1s 9ms/step - loss: 14342555.0000 - val_loss: 10722421.0000
Epoch 2/100
31/31 [==============================] - 0s 4ms/step - loss: 3160960.5000 - val_loss: 69603.5312
Epoch 3/100
 1/31 [..............................] - ETA: 0s - loss: 282776.2500

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



31/31 [==============================] - 0s 4ms/step - loss: 327624.0312 - val_loss: 71994.8672
Epoch 4/100
31/31 [==============================] - 0s 5ms/step - loss: 212168.4375 - val_loss: 43513.9375
Epoch 5/100
31/31 [==============================] - 0s 5ms/step - loss: 192454.4062 - val_loss: 38790.1562
Epoch 6/100
31/31 [==============================] - 0s 4ms/step - loss: 211392.1406 - val_loss: 31434.5391
Epoch 7/100
31/31 [==============================] - 0s 4ms/step - loss: 181519.3438 - val_loss: 29275.9688
Epoch 8/100
31/31 [==============================] - 0s 4ms/step - loss: 199308.8594 - val_loss: 36966.7812
Epoch 9/100
31/31 [==============================] - 0s 5ms/step - loss: 199422.5000 - val_loss: 21034.3145
Epoch 10/100
31/31 [==============================] - 0s 5ms/step - loss: 185421.0000 - val_loss: 16678.4414
Epoch 11/100
31/31 [==============================] - 0s 4ms/step - loss: 176416.7812 - val_loss: 21624.4980
Epoch 12/100
31/31 [==================

In [ ]:
results.append(BBRIRES2)
pd.DataFrame(BBRIRES2, index=[0]).T

,0
Dataset,BBRI
ANN_Train_RMSE,80.983221
ANN_Test_RMSE,73.460837
ANN_Train_MAE,59.809168
ANN_Test_MAE,55.900216


In [ ]:
results2.append(BBRIRES2)
pd.DataFrame(BBRIRES2, index=[0]).T

,0
Dataset,BBRI
ANN_Train_RMSE,80.983221
ANN_Test_RMSE,73.460837
ANN_Train_MAE,59.809168
ANN_Test_MAE,55.900216


In [ ]:
BBCARES2=process_dataset(bbca, 'BBCA')

Best: 1727.576171875 using {'batch_size': 64, 'epochs': 150, 'learning_rate': 0.01}
Epoch 1/150
16/16 [==============================] - 1s 24ms/step - loss: 16447578.0000 - val_loss: 14994871.0000
Epoch 2/150
12/16 [=====================>........] - ETA: 0s - loss: 10169879.0000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



16/16 [==============================] - 0s 12ms/step - loss: 8338335.0000 - val_loss: 110336.9609
Epoch 3/150
16/16 [==============================] - 0s 8ms/step - loss: 991044.7500 - val_loss: 542824.2500
Epoch 4/150
16/16 [==============================] - 0s 8ms/step - loss: 460176.1250 - val_loss: 117324.4688
Epoch 5/150
16/16 [==============================] - 0s 14ms/step - loss: 258390.0938 - val_loss: 62562.3164
Epoch 6/150
16/16 [==============================] - 0s 10ms/step - loss: 235154.6250 - val_loss: 62901.0898
Epoch 7/150
16/16 [==============================] - 0s 11ms/step - loss: 268111.2812 - val_loss: 44718.3008
Epoch 8/150
16/16 [==============================] - 0s 10ms/step - loss: 228320.5000 - val_loss: 40548.5977
Epoch 9/150
16/16 [==============================] - 0s 11ms/step - loss: 220483.9219 - val_loss: 35342.6758
Epoch 10/150
16/16 [==============================] - 0s 13ms/step - loss: 221293.4531 - val_loss: 30807.2441
Epoch 11/150
16/16 [========

In [ ]:
results2.append(BBCARES2)
pd.DataFrame(BBCARES2, index=[0]).T

,0
Dataset,BBCA
ANN_Train_RMSE,45.068448
ANN_Test_RMSE,41.624103
ANN_Train_MAE,32.591083
ANN_Test_MAE,30.959485


In [ ]:
BBNIRES2=process_dataset(bbni, 'BBNI')

Best: 846.2816772460938 using {'batch_size': 16, 'epochs': 100, 'learning_rate': 0.005}
Epoch 1/100
62/62 [==============================] - 1s 7ms/step - loss: 10680500.0000 - val_loss: 116875.2109
Epoch 2/100
46/62 [=====================>........] - ETA: 0s - loss: 329809.1250

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



62/62 [==============================] - 0s 4ms/step - loss: 319237.6562 - val_loss: 66058.8281
Epoch 3/100
62/62 [==============================] - 0s 3ms/step - loss: 279174.8125 - val_loss: 76716.9062
Epoch 4/100
62/62 [==============================] - 0s 4ms/step - loss: 255887.7188 - val_loss: 41592.4297
Epoch 5/100
62/62 [==============================] - 0s 4ms/step - loss: 268133.5938 - val_loss: 39757.2266
Epoch 6/100
62/62 [==============================] - 0s 3ms/step - loss: 243384.2969 - val_loss: 29830.4492
Epoch 7/100
62/62 [==============================] - 0s 3ms/step - loss: 223634.9844 - val_loss: 19168.5391
Epoch 8/100
62/62 [==============================] - 0s 4ms/step - loss: 215347.3438 - val_loss: 12819.4902
Epoch 9/100
62/62 [==============================] - 0s 3ms/step - loss: 232005.9844 - val_loss: 12968.4287
Epoch 10/100
62/62 [==============================] - 0s 3ms/step - loss: 225965.5781 - val_loss: 8365.9756
Epoch 11/100
62/62 [====================

In [ ]:
results2.append(BBNIRES2)
pd.DataFrame(BBNIRES2, index=[0]).T

,0
Dataset,BBNI
ANN_Train_RMSE,41.312902
ANN_Test_RMSE,40.284385
ANN_Train_MAE,31.835203
ANN_Test_MAE,31.318224


In [ ]:
resultsall2=pd.DataFrame(results2).T
resultsall2

,0,1,2,3
Dataset,BMRI,BBRI,BBCA,BBNI
ANN_Train_RMSE,37.259705,80.983221,45.068448,41.312902
ANN_Test_RMSE,34.430958,73.460837,41.624103,40.284385
ANN_Train_MAE,27.342167,59.809168,32.591083,31.835203
ANN_Test_MAE,25.485461,55.900216,30.959485,31.318224
